# Traffic Light Detection

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from time import time
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

%matplotlib inline

## CNN

In [38]:
img_width, img_height = 64, 64

input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, 3, 3, input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [17]:
# K.image_dim_ordering()

### Train Model

In [40]:
batch_size = 32
epochs = 5

In [41]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [42]:
training_set = train_datagen.flow_from_directory('data/train_dataset',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2600 images belonging to 1 classes.


In [43]:
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy',
    metrics=['binary_accuracy'])

In [44]:
%%time

model.fit(
    training_set,
    epochs=epochs)

Epoch 1/5
82/82 [==============================] - 137s 2s/step - loss: 0.0421 - binary_accuracy: 0.9977
Epoch 2/5
82/82 [==============================] - 83s 1s/step - loss: 2.2974e-05 - binary_accuracy: 1.0000
Epoch 3/5
82/82 [==============================] - 86s 1s/step - loss: 2.7151e-06 - binary_accuracy: 1.0000
Epoch 4/5
82/82 [==============================] - 84s 1s/step - loss: 4.6144e-06 - binary_accuracy: 1.0000
Epoch 5/5
82/82 [==============================] - 80s 981ms/step - loss: 2.5723e-06 - binary_accuracy: 1.0000
Wall time: 7min 52s


### Save Model

In [46]:
model.save_weights('Tlights' + '.h5') 

In [47]:
model_json = model.to_json()

In [49]:
with open('Tlights' + '.json', "w") as json_file:
    json_file.write(model_json)

### Test Model

In [52]:
test_datagen = ImageDataGenerator(
    rescale=1./255)

test_set = train_datagen.flow_from_directory(
    'data/test_dataset',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 400 images belonging to 1 classes.


In [53]:
%%time

i = 0
for images in test_set:
    print(images[0].shape)
    predictions = model.predict(images[0])
    i += 1
    if i > 0:
        break

(32, 64, 64, 3)
1/1 [==============================] - 0s 31ms/step
Wall time: 1.62 s


In [54]:
test_loss,test_acc = model.evaluate(test_set)
print('Test Accuracy:', test_acc)

13/13 [==============================] - 18s 1s/step - loss: 3.0945e-08 - binary_accuracy: 1.0000
Test Accuracy: 1.0
